# BEST BOOKS CHALLENGE

In [1]:
import pandas as pd

In [2]:
#Keep running untill 5000 items are scrapped
trial = 0
while True:
    try:
        import scrapper
    except:
        trial +=1
        print("\nNo of trials: "+str(trial))
    
    df = pd.read_csv('best-books.csv')
    next_row = df.shape[0]

    if next_row >= 5000:
        break
    

====================page(3)====================


## Load the csv file below

In [ ]:
df = pd.read_csv('best-books.csv')
print("\nData Shape: ",df.shape)
print(df.head())